In [ ]:
# Блок 1: Імпорт бібліотек та налаштування
import json
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import torch
import torch.nn as nn
from torchvision import transforms
import numpy as np

# Налаштування відображення
plt.rcParams['figure.figsize'] = [12, 8]
sns.set_style("whitegrid")

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Блок 2: Завантаження результатів тренування
print("Завантаження результатів тренування")

results_files = glob.glob('results/transfer_history_*.json')
print(f"Знайдено {len(results_files)} файлів результатів")

results_data = []
for file in results_files:
    with open(file, 'r') as f:
        data = json.load(f)
        results_data.append(data)

df = pd.DataFrame(results_data)
print("\nЗавантажені експерименти:")
for i, row in df.iterrows():
    print(f"  {row['model_name']} | {row['mode']} | {row['best_val_accuracy']:.2f}%")

In [ ]:
# Блок 3: Порівняльна таблиця результатів
print("ПОРІВНЯЛЬНА ТАБЛИЦЯ РЕЗУЛЬТАТІВ")
print("=" * 50)

comparison_table = df[['model_name', 'mode', 'best_val_accuracy']].copy()
comparison_table = comparison_table.sort_values('best_val_accuracy', ascending=False)

print(comparison_table.to_string(index=False))

In [ ]:
# Блок 4: Візуалізація порівняння точності
plt.figure(figsize=(10, 6))
sns.barplot(data=df, x='model_name', y='best_val_accuracy', hue='mode')
plt.title('Порівняння точності моделей')
plt.ylabel('Точність (%)')
plt.xlabel('Модель')
plt.xticks(rotation=45)
plt.legend(title='Режим')
plt.tight_layout()
plt.show()

In [ ]:
# Блок 5: Аналіз кривих навчання
print("АНАЛІЗ КРИВИХ НАВЧАННЯ")
print("=" * 40)

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.ravel()

for i, (idx, row) in enumerate(df.iterrows()):
    history = row['training_history']
    
    epochs = range(1, len(history['train_loss']) + 1)
    
    axes[i].plot(epochs, history['train_loss'], label='Train Loss', color='blue', linewidth=2)
    axes[i].plot(epochs, history['val_loss'], label='Val Loss', color='red', linewidth=2)
    axes[i].set_title(f"{row['model_name']} - {row['mode']}")
    axes[i].set_xlabel('Епоха')
    axes[i].set_ylabel('Loss')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Блок 6: Аналіз кривих точності
print("АНАЛІЗ КРИВИХ ТОЧНОСТІ")
print("=" * 40)

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.ravel()

for i, (idx, row) in enumerate(df.iterrows()):
    history = row['training_history']
    
    epochs = range(1, len(history['train_acc']) + 1)
    
    axes[i].plot(epochs, history['train_acc'], label='Train Accuracy', color='blue', linewidth=2)
    axes[i].plot(epochs, history['val_acc'], label='Val Accuracy', color='red', linewidth=2)
    axes[i].set_title(f"{row['model_name']} - {row['mode']}")
    axes[i].set_xlabel('Епоха')
    axes[i].set_ylabel('Точність (%)')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Блок 7: Завантаження найкращої моделі
print("ЗАВАНТАЖЕННЯ НАЙКРАЩОЇ МОДЕЛІ")
print("=" * 40)

best_result = df.loc[df['best_val_accuracy'].idxmax()]
best_model_name = best_result['model_name']
best_mode = best_result['mode']

print(f"Найкраща модель: {best_model_name}")
print(f"Режим: {best_mode}")
print(f"Точність: {best_result['best_val_accuracy']:.2f}%")

# Завантажуємо найкращу модель
from models.transfer_models import get_model

best_model = get_model(best_model_name, num_classes=3, pretrained=True, mode=best_mode)
best_model.load_state_dict(torch.load('checkpoints/transfer_best.pth', map_location='cpu'))
best_model.eval()

print("Модель успішно завантажена!")

In [ ]:
# Блок 8: Функція для тестування моделі
def predict_image(model, image_path, class_names):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])
    
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0)
    
    with torch.no_grad():
        outputs = model(image_tensor)
        _, predicted = torch.max(outputs, 1)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)[0]
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    ax1.imshow(image)
    ax1.set_title(f"Зображення: {image_path.split('/')[-1]}")
    ax1.axis('off')
    
    colors = ['lightblue' if i != predicted.item() else 'lightgreen' for i in range(len(class_names))]
    bars = ax2.barh(class_names, probabilities.numpy(), color=colors)
    ax2.set_xlim(0, 1)
    ax2.set_title('Ймовірності класів')
    ax2.set_xlabel('Ймовірність')
    
    for bar, prob in zip(bars, probabilities):
        ax2.text(bar.get_width() + 0.01, bar.get_y() + bar.get_height()/2, 
                f'{prob:.3f}', va='center')
    
    plt.tight_layout()
    plt.show()
    
    return predicted.item(), probabilities.numpy()

In [ ]:
# Блок 9: Тестування на валідаційних зображеннях
print("ТЕСТУВАННЯ НА ВАЛІДАЦІЙНИХ ЗОБРАЖЕННЯХ")
print("=" * 45)

test_images = glob.glob('data/val/*/*.jpg')[:6]

class_names = ['cat', 'dog', 'wild']  # Замінити на реальні назви класів

for i, img_path in enumerate(test_images, 1):
    print(f"Тест {i}/{len(test_images)}: {img_path}")
    true_class = img_path.split('/')[-2]
    predicted_class, probs = predict_image(best_model, img_path, class_names)
    
    print(f"Справжній клас: {true_class}")
    print(f"Прогнозований клас: {class_names[predicted_class]}")
    print(f"Впевненість: {probs[predicted_class]:.3f}")
    print("-" * 50)

In [ ]:
# Блок 11: Порівняння Feature Extraction vs Fine-Tuning
print("ПОРІВНЯННЯ FEATURE EXTRACTION VS FINE-TUNING")
print("=" * 55)

fe_results = df[df['mode'] == 'feature_extraction']
ft_results = df[df['mode'] == 'fine_tuning']

comparison_summary = []

for model in df['model_name'].unique():
    fe_acc = fe_results[fe_results['model_name'] == model]['best_val_accuracy']
    ft_acc = ft_results[ft_results['model_name'] == model]['best_val_accuracy']
    
    if len(fe_acc) > 0 and len(ft_acc) > 0:
        improvement = ft_acc.values[0] - fe_acc.values[0]
        comparison_summary.append({
            'model': model,
            'feature_extraction': fe_acc.values[0],
            'fine_tuning': ft_acc.values[0],
            'improvement': improvement
        })

comparison_df = pd.DataFrame(comparison_summary)
print(comparison_df.to_string(index=False, float_format='%.2f'))

In [ ]:
# Блок 12: Висновки та рекомендації
print("ВИСНОВКИ ТА РЕКОМЕНДАЦІЇ")
print("=" * 40)

best_overall = df.loc[df['best_val_accuracy'].idxmax()]
best_fe = df[df['mode'] == 'feature_extraction'].loc[df['best_val_accuracy'].idxmax()]
best_ft = df[df['mode'] == 'fine_tuning'].loc[df['best_val_accuracy'].idxmax()]

print(f"Найкраща модель загалом: {best_overall['model_name']} ({best_overall['mode']})")
print(f"Найкраща точність: {best_overall['best_val_accuracy']:.2f}%")
print()
print(f"Найкраща Feature Extraction: {best_fe['model_name']} - {best_fe['best_val_accuracy']:.2f}%")
print(f"Найкращий Fine-Tuning: {best_ft['model_name']} - {best_ft['best_val_accuracy']:.2f}%")
print(f"Різниця: {best_ft['best_val_accuracy'] - best_fe['best_val_accuracy']:.2f}%")
print()

improvement = best_ft['best_val_accuracy'] - best_fe['best_val_accuracy']
if improvement > 3:
    print("РЕКОМЕНДАЦІЯ: Використовувати Fine-Tuning - значно краща точність")
elif improvement > 1:
    print("РЕКОМЕНДАЦІЯ: Fine-Tuning дає помітне покращення")
else:
    print("РЕКОМЕНДАЦІЯ: Feature Extraction достатньо ефективний")

print()
print("ЗВЕДЕНА РЕКОМЕНДАЦІЯ:")
print(f"1. Для максимальної точності: {best_overall['model_name']} + Fine-Tuning")
print(f"2. Для швидкого навчання: {best_fe['model_name']} + Feature Extraction")

In [ ]:
# Збереження результатів аналізу
print("ЗБЕРЕЖЕННЯ РЕЗУЛЬТАТІВ АНАЛІЗУ")
print("=" * 45)

analysis_results = {
    'best_model': best_overall['model_name'],
    'best_mode': best_overall['mode'],
    'best_accuracy': float(best_overall['best_val_accuracy']),
    'comparison_summary': comparison_df.to_dict('records'),
    'timestamp': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
}

with open('results/analysis_summary.json', 'w') as f:
    json.dump(analysis_results, f, indent=2)

print("Результати аналізу збережено в results/analysis_summary.json")
print("Аналіз завершено!")

In [ ]:
%pip install pandas seaborn matplotlib pillow scikit-learn torch torchvision

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ------ --------------------------------- 1.8/11.0 MB 9.8 MB/s eta 0:00:01
   ---------------- ----------------------- 4.5/11.0 MB 11.0 MB/s eta 0:00:01
   ------------------------- -------------- 7.1/11.0 MB 11.3 MB/s eta 0:00:01
   --------------------------------- ------ 9.2/11.0 MB 11.4 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 10.6 MB/s  0:00:01
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------- ----------------------------- 2.4/8.7 MB 11.9 MB/s eta 0:00:01
   --------------------- ------------------ 4.7/8.7 MB 11.9 MB/s eta 0:00:01
   --------------------------------- ------ 7.3/8.7 MB 11.9 MB/s eta 0:00:01
   ---------------------------------------- 8.7/8.7 MB 11.3 MB/s  0:00:00
Using cached pytz-2025.2-py2.py3-non